In [1]:
# Import libraries

import os
from web3 import Web3
from dotenv import load_dotenv
from web3.middleware import geth_poa_middleware
from eth_account import Account
from pathlib import Path
from getpass import getpass

In [2]:
load_dotenv()
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [3]:
account_one = Account.from_key(os.getenv("PRIVATE_KEY"))
with open(
    Path(
        "UTC--2021-09-24T16-16-58.347316300Z--d29a838deecb9ad3e924841052fed69944a6f0c2"
    )
) as keyfile:
    encrypted_key = keyfile.read()
    private_key = w3.eth.account.decrypt(
        encrypted_key, getpass("Enter keystore password: ")
    )
    account_two = Account.from_key(private_key)

Enter keystore password:  ···


In [4]:
def create_raw_tx(account, recipient, amount):
    gasEstimate = w3.eth.estimateGas(
        {"from": account.address, "to": recipient.address, "value": amount}
    )
    return {
        "from": account.address,
        "to": recipient.address,
        "value": amount,
        "gasPrice": w3.eth.gasPrice,
        "gas": gasEstimate,
        "nonce": w3.eth.getTransactionCount(account.address),
    }

In [5]:
def send_tx(account, recipient, amount):
    tx = create_raw_tx(account, recipient, amount)
    signed_tx = account.sign_transaction(tx)
    result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
    print(result.hex())
    return result.hex()

In [6]:
print(account_one.address)
print(account_two.address)

0x0Cce5FBfB0C25AE0F40061F50AA491212Fa52a54
0xd29a838DEeCB9aD3E924841052FeD69944A6f0c2


In [7]:
send_tx(account_one,account_two,10000000000000000000000000)

ValueError: {'code': -32000, 'message': 'insufficient funds for gas * price + value'}